In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy

import k3d
import yaml
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm
import json

import warnings

from joblib import Parallel, delayed
from collections import defaultdict

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-185k5w31 because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '10'
os.environ['OMP_NUM_THREADS'] = '10'
os.environ['MKL_NUM_THREADS'] = '10'

In [4]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 0.5

            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.matplotlib_color_maps.coolwarm_r, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [5]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [7]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_point_patches
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE
from sharpf.utils.abc_utils.abc.feature_utils import compute_features_nbhood, remove_boundary_features, get_curves_extents
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.geometry import mean_mmd
import sharpf.data.data_smells as smells


In [8]:
with ABCChunk(['/data/abc/abc_0022_obj_v00.7z', '/data/abc/abc_0022_feat_v00.7z']) as data_holder:
    item = data_holder.get('00220090_ec51899db1f5298d674b3205_037')

In [9]:
item.item_id

'00220090_ec51899db1f5298d674b3205_037'

In [10]:
# mesh, vertex_normals, vertex_normal_indices = trimesh_load(item.obj)
mesh, _, _ = trimesh_load(item.obj)

features = yaml.load(item.feat, Loader=yaml.Loader)

In [11]:
mesh2 = mesh.copy()
mesh2._validate = True

In [12]:
mesh2.process()

<trimesh.Trimesh(vertices.shape=(14474, 3), faces.shape=(28944, 3))>

In [13]:
mesh

<trimesh.Trimesh(vertices.shape=(14474, 3), faces.shape=(28944, 3))>

In [14]:
mesh.is_watertight

True

In [15]:
for i in range(len(features['surfaces'])):
    print(i, np.array(np.unique(mesh.faces[features['surfaces'][i]['face_indices']]) == 
                      np.sort(features['surfaces'][i]['vert_indices'])).all()
         )

0 False
1 False
2 False
3 False
4 True
5 True


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
with open('/code/scripts/data_scripts/configs/pointcloud_datasets/dataset_config_high_res_clean.json') as config_file:
    config = json.load(config_file)

config['neighbourhood']['max_patches_per_mesh'] = 10000
# config['sampling']['resolution_3d'] = 0.05
# config['smell_deviating_resolution'] = {
#     'resolution_3d': 0.05,
#     'resolution_deviation_tolerance': 0.05 / 2
# }
config['noise'] = {
    'type': 'isotropic_gaussian',
    'scale': 0.0,
}
config

{'shape_fabrication_extent': 10.0,
 'short_curve_quantile': 0.25,
 'base_n_points_per_short_curve': 8,
 'base_resolution_3d': 0.125,
 'neighbourhood': {'type': 'random_euclidean_sphere',
  'max_patches_per_mesh': 10000,
  'n_vertices': None,
  'centroid': None,
  'centroid_mode': 'poisson_disk',
  'radius_base': 10.0,
  'radius_delta': 0.0,
  'geodesic_patches': True,
  'radius_scale_mode': 'no_scale'},
 'sampling': {'type': 'poisson_disk',
  'n_points': 4096,
  'resolution_3d': 0.02,
  'crop_center': True},
 'noise': {'type': 'isotropic_gaussian', 'scale': 0.0},
 'annotation': {'type': 'surface_based_aabb', 'distance_upper_bound': 1.0},
 'smell_coarse_surfaces_by_num_edges': {'num_edges_threshold': 8},
 'smell_coarse_surfaces_by_angles': {'max_angle_threshold_degrees': 10.0},
 'smell_deviating_resolution': {'resolution_3d': 0.02,
  'resolution_deviation_tolerance': 0.01},
 'smell_sharpness_discontinuities': {},
 'smell_bad_face_sampling': {'min_points_per_face': 0.02,
  'max_points_pe

In [17]:
shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)
short_curve_quantile = config.get('short_curve_quantile', 0.05)

nbhood_extractor = load_func_from_config(NBHOOD_BY_TYPE, config['neighbourhood'])
sampler = load_func_from_config(SAMPLER_BY_TYPE, config['sampling'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])


smell_coarse_surfaces_by_num_edges = smells.SmellCoarseSurfacesByNumEdges.from_config(config['smell_coarse_surfaces_by_num_edges'])
smell_coarse_surfaces_by_angles = smells.SmellCoarseSurfacesByAngles.from_config(config['smell_coarse_surfaces_by_angles'])
smell_deviating_resolution = smells.SmellDeviatingResolution.from_config(config['smell_deviating_resolution'])
smell_sharpness_discontinuities = smells.SmellSharpnessDiscontinuities.from_config(config['smell_sharpness_discontinuities'])
smell_bad_face_sampling = smells.SmellBadFaceSampling.from_config(config['smell_bad_face_sampling'])
smell_mesh_self_intersections = smells.SmellMeshSelfIntersections()


In [18]:

def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    mesh = mesh.apply_scale(least_len_mm / least_len)

    return mesh


In [19]:
mesh = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                      short_curve_quantile=short_curve_quantile,
                      n_points_per_short_curve=base_n_points_per_short_curve)

In [20]:
np.max(mesh.bounding_box.extents)

2.9233012144397343

In [21]:
smell_mesh_self_intersections.run(mesh)

False

In [22]:
surf_vert_indices = np.array(features['surfaces'][0]['vert_indices'])
surf_verts = mesh.vertices[surf_vert_indices]
surf_vert_indices2 = np.array(features['surfaces'][0]['vert_indices'])
surf_verts2 = mesh.vertices[surf_vert_indices]


display_sharpness(
    mesh, plot_meshvert=True, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, 
    sharpvert_psize=1. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples=None, samples_psize=1. * sampler.resolution_3d,
)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

Having surface area $S$, sampling resolution $r$ (= mean distance between points), and considering each point as occupying a surface $\pi r^2 / 4$, we have the number of points as:
$$
n = S / \pi r^2 / 4
$$

In [24]:
n_points = np.ceil(mesh.area / (np.pi * sampler.resolution_3d ** 2 / 4)).astype(int)
mesh.area, n_points

(17.25589091593374, 54928)

In [52]:
from sharpf.data.point_samplers import SamplerFunc


class PoissonDiskSampler(SamplerFunc):
    """Sample using the Poisson-Disk-Sampling of a mesh
    based on "Parallel Poisson Disk Sampling with Spectrum
    Analysis on Surface". (Implementation by fwilliams) """
    # https://github.com/marmakoide/mesh-blue-noise-sampling/blob/master/mesh-sampling.py
    def __init__(self, n_points, resolution_3d, make_n_points='crop_center'):
        self.make_n_points = make_n_points
        super().__init__(n_points, resolution_3d)

    @classmethod
    def from_config(cls, config):
        return cls(config['n_points'], config['resolution_3d'], config['make_n_points'])

    def _make_dense_mesh(self, mesh, extra_points_factor=10, point_split_factor=4):
        # Intuition: take 10x the number of needed n_points,
        # keep in mind that each call to `igl.upsample` generates 4x the points,
        # then compute the upsampling factor K from the relation:
        # 4^K n = 10 n_points
        upsampling_factor = np.ceil(
            np.log(self.n_points * extra_points_factor / len(mesh.vertices)) /
            np.log(point_split_factor)
        ).astype(int)

        # Generate very dense subdivision samples on the mesh (v, f, n)
        # for _ in range(upsampling_factor):
        #     mesh = mesh.subdivide()
        dense_points, dense_faces = igl.upsample(mesh.vertices, mesh.faces, upsampling_factor)

        # compute vertex normals by pushing to trimesh
        dense_mesh = trimesh.base.Trimesh(vertices=dense_points, faces=dense_faces, process=False, validate=False)
        return dense_mesh

    def sample(self, mesh, centroid=None):
        # check that the patch will not crash the upsampling function
        FF, FFi = igl.triangle_triangle_adjacency(mesh.faces)
        if (FF[FFi == -1] != -1).any() or (FFi[FF == -1] != -1).any():
            raise DataGenerationException('Mesh patch has issues and breaks the upsampling!')

        dense_mesh = self._make_dense_mesh(mesh, extra_points_factor=40, point_split_factor=4)
        dense_points = np.array(dense_mesh.vertices, order='C')
        dense_normals = np.array(dense_mesh.vertex_normals, order='C')
        dense_faces = np.array(dense_mesh.faces)

        # Downsample v_dense to be from a blue noise distribution:
        #
        # `points` is a downsampled version of `dense_points` where points are separated by approximately
        # `radius` distance, use_geodesic_distance indicates that the distance should be measured on the mesh.
        #
        # `normals` are the corresponding normals of `points`

        # require a little bit extra points as PDS get you sometimes fewer than requested
        required_points = int(1.1 * self.n_points)
        points, normals = pcu.sample_mesh_poisson_disk(
            dense_points, dense_faces, dense_normals,
            required_points, radius=self.resolution_3d, use_geodesic_distance=True)

        # ensure that we are returning exactly n_points
        if self.make_n_points == 'crop_center':
            centroid = np.mean(points, axis=1, keepdims=True) if centroid is None else centroid
            return_idx = np.argsort(np.linalg.norm(points - centroid, axis=1))[:self.n_points]
            
        elif self.make_n_points == 'sample':
            return_idx = np.random.choice(np.arange(len(points)), size=self.n_points, replace=False)
            
        else:
            assert None is self.make_n_points
            return_idx = np.arange(len(points))

        points, normals = points[return_idx], normals[return_idx]
        return points, normals


In [53]:
sampler = PoissonDiskSampler.from_config({
    'n_points': n_points,
    'resolution_3d': 0.02,
    'make_n_points': None
})

In [54]:
whole_model_points, whole_model_normals = sampler.sample(mesh)

In [58]:
display_sharpness(
    mesh, plot_meshvert=False, meshvert_psize=sampler.resolution_3d,
    samples=whole_model_points, samples_psize=1. * sampler.resolution_3d,
)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [61]:
config_n_points = config['sampling']['n_points']

full_model_resolution_discount = 4.0
nbhood_extractor.radius_base = np.sqrt(config_n_points) * 0.5 * sampler.resolution_3d / full_model_resolution_discount

nbhood_extractor.index(mesh)

full_model_resolution_discount = 1.0
nbhood_extractor.radius_base = np.sqrt(config_n_points) * 0.5 * sampler.resolution_3d / full_model_resolution_discount


In [62]:
len(nbhood_extractor.centroids_cache)

75

In [63]:
has_smell_mismatching_surface_annotation = any([
    np.array(np.unique(mesh.faces[surface['face_indices']]) != np.sort(surface['vert_indices'])).all()
    for surface in features['surfaces']
])

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
LARGEST_PROCESSABLE_MESH_VERTICES = 20000


def compute_patches(
        patch_idx,
        whole_model_points,
        mesh,
        features,
        nbhood_extractor,
        sampler,
        noiser,
        annotator,
        smell_coarse_surfaces_by_num_edges,
        smell_coarse_surfaces_by_angles,
        smell_deviating_resolution,
        smell_bad_face_sampling,
        smell_sharpness_discontinuities):


    global LARGEST_PROCESSABLE_MESH_VERTICES

    nbhood_extractor.current_patch_idx = patch_idx

    # extract neighbourhood
    try:
        nbhood, mesh_vertex_indexes, mesh_face_indexes, scaler = nbhood_extractor.get_nbhood()
        if len(nbhood.vertices) > LARGEST_PROCESSABLE_MESH_VERTICES:
            raise DataGenerationException('Too large number of vertices in crop: {}'.format(len(nbhood.vertices)))
    except DataGenerationException as e:
        eprint_t(str(e))
        return None
    centroid = nbhood_extractor.centroid

    has_smell_coarse_surfaces_by_num_edges = smell_coarse_surfaces_by_num_edges.run(mesh, mesh_face_indexes, features)
    has_smell_coarse_surfaces_by_angles = smell_coarse_surfaces_by_angles.run(mesh, mesh_face_indexes, features)

    # create annotations: condition the features onto the nbhood
    nbhood_features = compute_features_nbhood(mesh, features, mesh_face_indexes,
                                              mesh_vertex_indexes=mesh_vertex_indexes)

    # remove vertices lying on the boundary (sharp edges found in 1 face only)
    nbhood_features = remove_boundary_features(nbhood, nbhood_features, how='edges')

    # sample the neighbourhood to form a point patch
#     try:
#         points, normals = sampler.sample(nbhood, centroid=nbhood_extractor.centroid)
#     except DataGenerationException as e:
#         eprint_t(str(e))
#         return None

    distance_sq, face_indexes, _ = igl.point_mesh_squared_distance(
        whole_model_points,
        nbhood.vertices,
        nbhood.faces)
    indexes = np.where(distance_sq < sampler.resolution_3d / 100)[0]
    points, normals = whole_model_points[indexes], nbhood.face_normals[face_indexes[indexes]]

    has_smell_deviating_resolution = smell_deviating_resolution.run(points)
    has_smell_bad_face_sampling = smell_bad_face_sampling.run(nbhood, points)

    # create a noisy sample
    noisy_points = noiser.make_noise(points, normals)
    
    try:
        distances, directions, has_sharp = annotator.annotate(nbhood, nbhood_features, noisy_points)
    except DataGenerationException as e:
        eprint_t(str(e))
        return None 

    has_smell_sharpness_discontinuities = smell_sharpness_discontinuities.run(noisy_points, distances)

    num_sharp_curves = len([curve for curve in nbhood_features['curves'] if curve['sharp']])
    num_surfaces = len(nbhood_features['surfaces'])
    patch = {
        'points': np.array(noisy_points).astype(np.float64),
        'normals': np.array(normals).astype(np.float64),
        'distances': np.array(distances).astype(np.float64),
        'directions': np.array(directions).astype(np.float64),
        'orig_vert_indices': np.array(mesh_vertex_indexes).astype(np.int32),
        'orig_face_indexes': np.array(mesh_face_indexes).astype(np.int32),
        'has_sharp': has_sharp,
        'num_sharp_curves': num_sharp_curves,
        'num_surfaces': num_surfaces,
        'has_smell_coarse_surfaces_by_num_faces': has_smell_coarse_surfaces_by_num_edges,
        'has_smell_coarse_surfaces_by_angles': has_smell_coarse_surfaces_by_angles,
        'has_smell_deviating_resolution': has_smell_deviating_resolution,
        'has_smell_sharpness_discontinuities': has_smell_sharpness_discontinuities,
        'has_smell_bad_face_sampling': has_smell_bad_face_sampling,
#         'has_smell_mismatching_surface_annotation': has_smell_mismatching_surface_annotation,
        'nbhood': nbhood,
        'indexes': indexes
    }
    return patch

In [67]:
patch = compute_patches(
    0,
    whole_model_points,
    mesh,
    features,
    nbhood_extractor,
    sampler,
    noiser,
    annotator,
    smell_coarse_surfaces_by_num_edges,
    smell_coarse_surfaces_by_angles,
    smell_deviating_resolution,
    smell_bad_face_sampling,
    smell_sharpness_discontinuities
)

In [68]:
display_sharpness(
    patch['nbhood'], plot_meshvert=False, meshvert_psize=sampler.resolution_3d,
#     sharp_vert=nbhood.vertices, 
#     sharpvert_psize=10. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples=patch['points'], samples_psize=1. * sampler.resolution_3d,
    samples_distances=patch['distances']
)

Output()

In [69]:
parallel = Parallel(n_jobs=16, backend='multiprocessing', verbose=100)
delayed_iterable = (delayed(compute_patches)(
    patch_idx,
    whole_model_points,
    mesh,
    features,
    nbhood_extractor,
    sampler,
    noiser,
    annotator,
    smell_coarse_surfaces_by_num_edges,
    smell_coarse_surfaces_by_angles,
    smell_deviating_resolution,
    smell_bad_face_sampling,
    smell_sharpness_discontinuities)
for patch_idx in range(nbhood_extractor.n_patches_per_mesh))
result = parallel(delayed_iterable)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 tasks      | elapsed:    1.7s


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:    3.3s


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[Parallel(n_jobs=16)]: Done   3 tasks      | elapsed:    3.7s


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/joblib/parallel.py:722: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/user/miniconda/envs/py36/lib/

[Parallel(n_jobs=16)]: Done   4 tasks      | elapsed:   13.4s
[Parallel(n_jobs=16)]: Done   5 tasks      | elapsed:   14.2s
[Parallel(n_jobs=16)]: Done   6 tasks      | elapsed:   14.3s
[Parallel(n_jobs=16)]: Done   7 tasks      | elapsed:   14.4s
[Parallel(n_jobs=16)]: Done   8 tasks      | elapsed:   14.5s
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:   16.0s
[Parallel(n_jobs=16)]: Done  10 tasks      | elapsed:   16.8s
[Parallel(n_jobs=16)]: Done  11 tasks      | elapsed:   18.5s
[Parallel(n_jobs=16)]: Done  12 tasks      | elapsed:   19.1s
[Parallel(n_jobs=16)]: Done  13 tasks      | elapsed:   19.1s
[Parallel(n_jobs=16)]: Done  14 tasks      | elapsed:   19.3s
[Parallel(n_jobs=16)]: Done  15 tasks      | elapsed:   20.1s
[Parallel(n_jobs=16)]: Done  16 tasks      | elapsed:   20.2s
[Parallel(n_jobs=16)]: Done  17 tasks      | elapsed:   20.5s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   21.1s
[Parallel(n_jobs=16)]: Done  19 tasks      | elapsed:   21.5s
[Paralle

In [73]:
whole_model_distances = np.ones(len(whole_model_points)) * np.inf
whole_model_directions = np.ones( (len(whole_model_points), 3) ) * np.inf

In [74]:
for patch in tqdm(result):
    distances = patch['distances']
    directions = patch['directions']
    indexes = patch['indexes']

    assign_mask = whole_model_distances[indexes] > distances
    whole_model_distances[indexes[assign_mask]] = distances[assign_mask]
    whole_model_directions[indexes[assign_mask]] = directions[assign_mask]

100%|██████████| 75/75 [00:00<00:00, 3795.38it/s]


In [72]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=whole_model_points, samples_distances=whole_model_distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=None)

Output()

In [74]:
whole_patches = []

for patch in tqdm(result):
    whole_patch = deepcopy(patch)
    whole_patch['points'] = whole_patch['points'].ravel()
    whole_patch['normals'] = whole_patch['normals'].ravel()
    whole_patch['distances'] = whole_model_distances[patch['indexes']].ravel()
    whole_patch['directions'] = whole_model_directions[patch['indexes'], :].ravel()
    whole_patch['has_smell_mismatching_surface_annotation'] = has_smell_mismatching_surface_annotation
    whole_patch['item_id'] = item.item_id
    whole_patch.pop('nbhood')
    whole_patch.pop('indexes')
    whole_patch['indexes_in_whole'] = patch['indexes']
    whole_patches.append(whole_patch)

100%|██████████| 1101/1101 [00:00<00:00, 1549.59it/s]


In [80]:
idx = np.random.randint(len(result))

patch = whole_patches[idx]

display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=patch['points'].reshape((-1, 3)), samples_distances=patch['distances'],
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=patch['directions'].reshape((-1, 3)))

Vectors(colors=array([], dtype=uint32), head_color=255, id=140391723338664, line_width=0.0025, model_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32), origin_color=255, origins=array([[-0.91163605,  3.9049733 ,  1.3942498 ],
       [-1.180367  ,  4.291388  ,  1.1762593 ],
       [-1.0638735 ,  4.481517  ,  1.2829361 ],
       ...,
       [-0.9122167 ,  4.2156377 ,  1.3947088 ],
       [-1.240742  ,  4.2024865 ,  1.1124586 ],
       [-0.79642254,  3.95896   ,  1.4630924 ]], dtype=float32), type='Vectors', vectors=array([[ 7.7517296e-04,  9.9052566e-01, -1.1432274e-03],
       [ 1.1846676e-03,  6.0411090e-01, -1.1222548e-03],
       [ 1.5020202e-03,  4.1398209e-01, -1.7682882e-03],
       ...,
       [ 1.1712146e-03,  6.7986125e-01, -1.7274305e-03],
       [ 1.1529304e-03,  6.9301242e-01, -1.0920764e-03],
       [ 6.9720688e-04,  9.3653888e-01, -1.3150685e-03]], dtype=float32))


Output()

In [83]:
WholePointCloudIO = io.HDF5IO({
    'points': io.VarFloat64('points'),
    'normals': io.VarFloat64('normals'),
    'distances': io.VarFloat64('distances'),
    'directions': io.VarFloat64('directions'),
    'indexes_in_whole': io.VarInt32('indexes_in_whole'),
    'item_id': io.AsciiString('item_id'),
    'orig_vert_indices': io.VarInt32('orig_vert_indices'),
    'orig_face_indexes': io.VarInt32('orig_face_indexes'),
    'has_sharp': io.Bool('has_sharp'),
    'num_sharp_curves': io.Int8('num_sharp_curves'),
    'num_surfaces': io.Int8('num_surfaces'),
    'has_smell_coarse_surfaces_by_num_faces': io.Bool('has_smell_coarse_surfaces_by_num_faces'),
    'has_smell_coarse_surfaces_by_angles': io.Bool('has_smell_coarse_surfaces_by_angles'),
    'has_smell_deviating_resolution': io.Bool('has_smell_deviating_resolution'),
    'has_smell_sharpness_discontinuities': io.Bool('has_smell_sharpness_discontinuities'),
    'has_smell_bad_face_sampling': io.Bool('has_smell_bad_face_sampling'),
    'has_smell_mismatching_surface_annotation': io.Bool('has_smell_mismatching_surface_annotation'),
},
len_label='has_sharp',
compression='lzf')

In [94]:
from functools import partial 
import h5py

def save_whole_patches(patches, filename):
    # turn a list of dicts into a dict of torch tensors:
    # default_collate([{'a': 'str1', 'x': np.random.normal()}, {'a': 'str2', 'x': np.random.normal()}])
    # Out[26]: {'a': ['str1', 'str2'], 'x': tensor([0.4252, 0.1414], dtype=torch.float64)}
    collate_fn = partial(io.collate_mapping_with_io, io=WholePointCloudIO)
    patches = collate_fn(patches)

    with h5py.File(filename, 'w') as f:
        for key in ['points', 'normals', 'distances', 'directions', 'indexes_in_whole']:
            WholePointCloudIO.write(f, key, patches[key])
        WholePointCloudIO.write(f, 'item_id', patches['item_id'])
        WholePointCloudIO.write(f, 'orig_vert_indices', patches['orig_vert_indices'])
        WholePointCloudIO.write(f, 'orig_face_indexes', patches['orig_face_indexes'])
        WholePointCloudIO.write(f, 'has_sharp', patches['has_sharp'].numpy().astype(np.bool))
        WholePointCloudIO.write(f, 'num_sharp_curves', patches['num_sharp_curves'].numpy())
        WholePointCloudIO.write(f, 'num_surfaces', patches['num_surfaces'].numpy())
        has_smell_keys = [key for key in WholePointCloudIO.datasets.keys()
                          if key.startswith('has_smell')]
        for key in has_smell_keys:
            PointCloudIO.write(f, key, patches[key].numpy().astype(np.bool))

In [95]:
save_whole_patches(whole_patches, '/logs/whole_{}.hdf5'.format(item.item_id))

In [6]:
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes
import sharpf.utils.abc_utils.hdf5.io_struct as io
import sharpf.data.datasets.sharpf_io as io 

In [9]:
dataset = Hdf5File('/logs/abc_0022_00220090_ec51899db1f5298d674b3205_037.hdf5',
                   io=io.WholePointCloudIO,
                   preload=PreloadTypes.LAZY,
                   labels='*')

In [15]:
idx = np.random.randint(len(dataset))

patch = dataset[idx]

display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=0.02,
                  samples=patch['points'].reshape((-1, 3)), samples_distances=patch['distances'],
                  samples_color=0x0000ff, samples_psize=0.02,
                  directions=patch['directions'].reshape((-1, 3)))

Vectors(colors=array([], dtype=uint32), head_color=255, id=140142132386504, line_width=0.0025, model_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32), origin_color=255, origins=array([[-0.29543132, -0.29372117, -0.28678182],
       [-0.29102084, -0.29204527, -0.3032786 ],
       [-0.28287807, -0.29605648, -0.28139922],
       ...,
       [-0.9756016 , -0.05885366,  0.05854423],
       [-0.98914415, -0.05885366,  0.03921641],
       [-0.9909573 , -0.05885366,  0.06072959]], dtype=float32), type='Vectors', vectors=array([[-4.1604769e-01,  2.3486714e-01, -4.1185358e-01],
       [-3.9317936e-01,  2.3319122e-01, -4.2210862e-01],
       [-4.2507905e-01,  2.3720245e-01, -4.2079392e-01],
       ...,
       [-1.9787207e-02,  0.0000000e+00,  1.4059661e-03],
       [-7.2520049e-03,  0.0000000e+00,  2.2054254e-04],
       [-4.3541389e-03,  6.9372619e-18,  3.0937864e-04]], dtype=float32))


Output()

In [163]:
def combine_predictions(predictions, patches):
    n_points = np.concatenate([
        patch['indexes_in_whole']
        for patch in patches]).max() + 1
    
    whole_model_points_pred = np.zeros((n_points, 3))
    whole_model_distances_pred = np.ones(n_points) * np.inf
    whole_model_directions_pred = np.ones( (n_points, 3) ) * np.inf
    
    for prediction, patch in zip(predictions, patches):
        distances = prediction['distances']
        directions = prediction['directions'].reshape((-1, 3))
        indexes = patch['indexes_in_whole']
        whole_model_points_pred[indexes] = patch['points'].reshape((-1, 3))

        assign_mask = whole_model_distances_pred[indexes] > distances
        whole_model_distances_pred[indexes[assign_mask]] = np.minimum(distances[assign_mask], 1.0)
        whole_model_directions_pred[indexes[assign_mask]] = directions[assign_mask]
        
    return whole_model_points_pred, whole_model_distances_pred, whole_model_directions_pred

In [165]:
whole_model_points_pred, whole_model_distances_pred, whole_model_directions_pred = combine_predictions(
    patches, patches)

In [164]:
patches = [patch for patch in dataset]

In [146]:
n_points = np.concatenate([
    patch['indexes_in_whole']
    for patch in dataset]).max() + 1

In [157]:
whole_model_points_pred = np.zeros((n_points, 3))
whole_model_distances_pred = np.ones(n_points) * np.inf
whole_model_directions_pred = np.ones( (n_points, 3) ) * np.inf

In [158]:
for patch in tqdm(dataset):
    distances = patch['distances'] + np.random.uniform(0.0, 0.1, size=len(patch['distances']))
    directions = patch['directions'].reshape((-1, 3))
    indexes = patch['indexes_in_whole']
    whole_model_points_pred[indexes] = patch['points'].reshape((-1, 3))

    assign_mask = whole_model_distances_pred[indexes] > distances
    whole_model_distances_pred[indexes[assign_mask]] = np.minimum(distances[assign_mask], 1.0)
    whole_model_directions_pred[indexes[assign_mask]] = directions[assign_mask]

100%|██████████| 1101/1101 [00:00<00:00, 1462.67it/s]


In [166]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=whole_model_points_pred, samples_distances=whole_model_distances_pred,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=None)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [ ]:
whole_model_points_pred

In [ ]:
whole_model_points = np.concatenate([
    patch['points']
    for patch in dataset
])
whole_model_points.shape

In [ ]:
whole_model_directions = np.concatenate([
    patch['directions']
    for patch in dataset
])
whole_model_directions.shape

In [ ]:
whole_model_distances = np.concatenate([
    patch['distances']
    for patch in dataset
])
whole_model_distances.shape

In [ ]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=0.02,
                  samples=whole_model_points, samples_distances=whole_model_distances,
                  samples_color=0x0000ff, samples_psize=0.02,
                  directions=None)